In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

import mysql.connector
# from sqlalchemy import create_engine

import re
import json

In [2]:
# DB설정
with open('./path.json') as file:
    json_obj = json.load(file)
    
con = mysql.connector.connect(host = json_obj['DB']['Host'],
                              port = json_obj['DB']['Port'],
                              database = 'capstone',
                              user = json_obj['DB']['User'],
                              password = str(json_obj['DB']['Password'])
)

In [3]:
cursor = con.cursor(dictionary=True)

In [4]:
cursor.execute('show Databases;')
result = cursor.fetchall()

In [5]:
pd.DataFrame(result)

,Database
0,capstone
1,information_schema
2,mysql
3,performance_schema
4,sys


In [19]:
apt = pd.read_csv("./data/실거래_가공/아파트_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
officetel = pd.read_csv("./data/실거래_가공/오피스텔_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
yeon = pd.read_csv('./data/실거래_가공/연립다세대_실거래_10년_가공.csv',index_col = 0, encoding='cp949')

/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_67949/1795912215.py:1: DtypeWarning: Columns (11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  apt = pd.read_csv("./data/실거래_가공/아파트_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_67949/1795912215.py:2: DtypeWarning: Columns (11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  officetel = pd.read_csv("./data/실거래_가공/오피스텔_실거래_10년_가공.csv",index_col = 0, encoding='cp949')
/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_67949/1795912215.py:3: DtypeWarning: Columns (11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  yeon = pd.read_csv('./data/실거래_가공/연립다세대_실거래_10년_가공.csv',index_col = 0, encoding='cp949')


In [20]:
print(len(apt))
print(len(officetel))
print(len(yeon))
apt.columns == officetel.columns # 컬럼 동일

print(apt.columns)
print(yeon.columns) # 건물명과 단지명 차이가 있음

2024191
447133
1073939
Index(['시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)', '월세(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용',
       '종전계약 보증금 (만원)', '종전계약 월세 (만원)', '법정동코드', '산', '토지고유번호'],
      dtype='object')
Index(['시군구', '번지', '본번', '부번', '건물명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)', '월세(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용',
       '종전계약 보증금 (만원)', '종전계약 월세 (만원)', '법정동코드', '산', '토지고유번호'],
      dtype='object')


In [8]:
import re
import json

re1 = re.compile("^(수원|고양|용인|창원|성남|청주|전주|천안|안산|안양|포항)")
re2 = re.compile("([읍면동]|[0-9]가|시장북로|세종로|남성로)$")


In [21]:
def parse(address, how='jibun'):
    '''
    주소를 받고 how 값에 따라 지번 또는 도로명 주소로 값을 return하는 메소드

    address : 주소
    how : 지번으로 parsing 또는 도로명으로 parsing ('jibun', 'road')
    '''
    parts = address.split()
    sido = ""
    sigungu = ""
    ilbangu = ""
    eupmyeondong = ""
    ri_roadname = ""
    for idx, part in enumerate(parts):
        if idx == 0:
            sido = part
            continue
        
        if part.endswith("시") or part.endswith("군"):
            sigungu = part
            continue
        
        if part.endswith("구"):
            if idx == 1:
                if len(part) >= 4 and re.search(re1, part):
                    sigungu = part[0:2] + "시"
                    ilbangu = part[2:]
                else:
                    sigungu = part
            else:
                ilbangu = part
            continue
        
        if re.search(re2,
                        part) and eupmyeondong == '' and how == 'jibun':
            eupmyeondong = part
            continue
            
        if part.endswith("리") and how == 'jibun':
            ri_roadname = part
            continue
        
        if (part.endswith('로') or part.endswith('길')
            ) and how == 'road' and ri_roadname == '':
            ri_roadname = part
            continue
        
        if re.search(r"([가-힣]{2,})(\([^0-9가-힣a-zA-Z]{1,}\))", part):
            ri_roadname = part
            continue

    return sido, sigungu, ilbangu, eupmyeondong, ri_roadname


In [27]:
t = apt.sample(50)

In [39]:
apt[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(apt['시군구'].iloc[i]) for i in tqdm(range(len(apt)))]
officetel[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(officetel['시군구'].iloc[i]) for i in tqdm(range(len(officetel)))]
yeon[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]] = [parse(yeon['시군구'].iloc[i]) for i in tqdm(range(len(yeon)))]

100%|██████████| 1073939/1073939 [00:06<00:00, 159364.71it/s]


In [37]:
apt[["sido", "sigungu", "ilbangu", "eupmyeondong", "ri_roadname"]]

    2024191
Name: ri_roadname, dtype: int64

In [40]:
apt['eupmyeondong'].value_counts()
apt.sample(5)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),...,종전계약 보증금 (만원),종전계약 월세 (만원),법정동코드,산,토지고유번호,sido,sigungu,ilbangu,eupmyeondong,ri_roadname
1885791,서울특별시 동작구 상도동,531,531.0,0.0,힐스테이트 상도 센트럴파크,전세,84.6099,202306,12,"75,000",...,"93,000",0.0,1159010200,1,1159010200105310000,서울특별시,동작구,,상도동,
1823896,서울특별시 관악구 신림동,1730,1730.0,0.0,신림푸르지오,전세,138.7400,202307,26,"65,000",...,0,0.0,1162010200,1,1162010200117300000,서울특별시,관악구,,신림동,
1242322,서울특별시 마포구 중동,40-12,40.0,12.0,현대,전세,84.9400,202012,8,"44,000",...,NaN,NaN,1144012600,1,1144012600100400012,서울특별시,마포구,,중동,
1141438,서울특별시 강남구 대치동,316,316.0,0.0,은마,전세,84.4300,202001,6,"57,000",...,NaN,NaN,1168010600,1,1168010600103160000,서울특별시,강남구,,대치동,
905331,서울특별시 성북구 장위동,316-2,316.0,2.0,"장월에스에이치빌1단지(국민임대,SH소유",월세,49.3300,201806,21,"1,792",...,NaN,NaN,1129013800,1,1129013800103160002,서울특별시,성북구,,장위동,


In [15]:
t

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),...,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원),법정동코드,산,토지고유번호,시,일반구,읍면동,리
0,0,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201301,11,"35,000",...,-,NaN,NaN,1168010300,1,1168010300106550002,0,0,0,0
1,0,655-2,655.0,2.0,개포2차현대아파트(220),월세,77.75,201301,29,"24,000",...,-,NaN,NaN,1168010300,1,1168010300106550002,0,0,0,0
2,0,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201303,4,"32,000",...,-,NaN,NaN,1168010300,1,1168010300106550002,0,0,0,0
3,0,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201309,14,"37,000",...,-,NaN,NaN,1168010300,1,1168010300106550002,0,0,0,0
4,0,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201309,28,"40,000",...,-,NaN,NaN,1168010300,1,1168010300106550002,0,0,0,0
5,0,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201310,17,"32,000",...,-,NaN,NaN,1168010300,1,1168010300106550002,0,0,0,0
6,0,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201311,27,"34,500",...,-,NaN,NaN,1168010300,1,1168010300106550002,0,0,0,0
7,0,658-1,658.0,1.0,개포6차우성아파트1동~8동,전세,79.97,201301,7,"30,000",...,-,NaN,NaN,1168010300,1,1168010300106580001,0,0,0,0
8,0,658-1,658.0,1.0,개포6차우성아파트1동~8동,전세,79.97,201301,14,"31,000",...,-,NaN,NaN,1168010300,1,1168010300106580001,0,0,0,0
9,0,658-1,658.0,1.0,개포6차우성아파트1동~8동,전세,79.97,201301,18,"29,000",...,-,NaN,NaN,1168010300,1,1168010300106580001,0,0,0,0


In [ ]:
def inserter(row, data_type, data_len):
    
    '''
    각 row 별로 insert 작업 수행

    row : DataFrame의 row
    jdbc : JDBC 객체
    data_type : 데이터 종류 (예 : '아파트-전세', '아파트-매매')
    data_len : 총 데이터 수
    '''
    insert_tuple = (
        None if pd.isna(row['토지고유번호']) else row['토지고유번호'],  # address_id
        row['시도'],
        row['시군구'],
        row['일반구'],
        row['읍면동'],
        row['리'],
        2 if '산' in str(row['번지']) else 1,  # 산여부(1:일반, 2:산)
        None if pd.isna(row['본번']) else str(row['본번']),  # num1
        None if pd.isna(row['부번']) else str(row['부번']),  # num2
        row['도로명'],  # road_name
        None if pd.isna(row['단지명']) else row['단지명'],  # apart_name
        '아파트',  # property_type(주택유형)
        None if pd.isna(row['층']) else str(int(row['층'])),  # floor_number(층 수)
        row['전용면적(㎡)'],  # exclusive_area(전용면적)
        str(row['계약날짜']),  # contract_date(계약날짜)
        str(int(row['거래금액(만원)'].replace(',', '')) * 10000),  # price(거래금액)
        None if pd.isna(row['거래유형']) else row['거래유형'],
        None if pd.isna(row['중개사소재지']) or row['중개사소재지'] == '-' else
        row['중개사소재지'],
        None if pd.isna(row['건축년도']) else str(int(row['건축년도'])),
        1,
        None if pd.isna(row['해제사유발생일']) else str(int(row['해제사유발생일'])), # 해제사유일
        row['cFlag'], # 해제사유 취소여부 플래그
        row['hash'] # 해시 값
        )
    index = row.name + 1
    try:
        if index % (data_len // 10) == 0:
            print(f'{data_type} : {index/data_len*100:.2f}% a_id INSERT 완료')
    except:
        pass
    

In [19]:
apt.sample(3)['시군구']


parse(apt.sample(1)['시군구'].iloc[0])




('서울특별시', '강동구', '', '상일동', '')

In [23]:
apt['번지']

0           655-2
1           655-2
2           655-2
3           655-2
4           655-2
            ...  
2024444       450
2024445    274-75
2024446    274-77
2024447        11
2024448      11-7
Name: 번지, Length: 2024191, dtype: object

In [ ]:
import re
import json


class AddressParser:
    '''
    주소를 각 명칭으로 나누거나
    토지고유번호를 매칭해주는 class
    '''
    __jibun_search_query = '''
        SELECT {}
        FROM addresses 
        WHERE sido = %s 
        AND sigungu = %s 
        AND ilbangu = %s 
        AND eupmyeondong = %s 
        AND ri = %s 
        AND is_mountain = %s 
        AND num1 = %s
        AND {}
    '''
    __road_search_query = '''
        SELECT {}
        FROM addresses 
        WHERE sido = %s 
        AND sigungu = %s 
        AND ilbangu = %s 
        AND road_name = %s
        AND road_num1 = %s
        AND road_num2 = %s
    '''
    __search_by_aid = '''
        SELECT {}
        FROM addresses
        WHERE id = %s
    '''

    def __init__(self):
        '''
        주소를 불러와
        sido, sigungu, ilbangu, eupmyeondong, ri_roadname으로 나누는 작업을 해주는 class
        '''
        self.re1 = re.compile("^(수원|고양|용인|창원|성남|청주|전주|천안|안산|안양|포항)")
        self.re2 = re.compile("([읍면동]|[0-9]가|시장북로|세종로|남성로)$")

    def parse(self, address, how='jibun'):
        '''
        주소를 받고 how 값에 따라 지번 또는 도로명 주소로 값을 return하는 메소드

        address : 주소
        how : 지번으로 parsing 또는 도로명으로 parsing ('jibun', 'road')
        '''
        parts = address.split()
        sido = ""
        sigungu = ""
        ilbangu = ""
        eupmyeondong = ""
        ri_roadname = ""
        for idx, part in enumerate(parts):
            if idx == 0:
                sido = part
                continue
            
            if part.endswith("시") or part.endswith("군"):
                sigungu = part
                continue
            
            if part.endswith("구"):
                if idx == 1:
                    if len(part) >= 4 and re.search(self.re1, part):
                        sigungu = part[0:2] + "시"
                        ilbangu = part[2:]
                    else:
                        sigungu = part
                else:
                    ilbangu = part
                continue
            
            if re.search(self.re2,
                         part) and eupmyeondong == '' and how == 'jibun':
                eupmyeondong = part
                continue
                
            if part.endswith("리") and how == 'jibun':
                ri_roadname = part
                continue
            
            if (part.endswith('로') or part.endswith('길')
                ) and how == 'road' and ri_roadname == '':
                ri_roadname = part
                continue
            
            if re.search(r"([가-힣]{2,})(\([^0-9가-힣a-zA-Z]{1,}\))", part):
                ri_roadname = part
                continue

        return sido, sigungu, ilbangu, eupmyeondong, ri_roadname

    def get_address_id(self,
                       addr,
                       http,
                       jdbc,
                       silent=False,
                       additional_columns=None):
        '''
        지번 또는 도로명 주소로 구성 되어있는 주소를 parsing하여 토지 고유번호 매칭.
        깔끔하게 구성한 주소여야 함.
        대량의 주소를 변경하는 경우로 예상하여 http나 jdbc 객체를 받게끔 정의

        addr : 주소
        http : urllib3 PoolManager 객체
        jdbc : JDBC 객체
        silent : 못 찾았을 때, 출력값 보여주기 여부
        additional_columns : addresses에서 id 값 뿐만 아니라 더 검색할 columns 값들
        '''
        search_columns = 'id, '
        if additional_columns is not None:
            for column in additional_columns:
                search_columns += column + ', '
        search_columns = search_columns[:-2]

        def search_road(addr, splitted_addr):
            '''
            주소 형태가 도로명 주소인 경우라고 판단 할 때, 매칭하는 코드
            먼저 postcodify에서 검색 후 없으면 addresses에서 검색

            addr : 주소
            splitted_addr : 분리된 주소
            '''
            number = addr.split()[-1]
            road_name = splitted_addr[-1] + ' ' + number

            number = number.split('-')
            num1 = number[0]
            num2 = number[1] if len(number) == 2 else 0

            r = http.request('GET',
                             'http://192.168.20.2/postcodify/lib/search.php',
                             fields={'q': addr})

            if r.status == 200:
                results = json.loads(r.data.decode('utf8'))['results']
                filtered = list(
                    filter(lambda x: x['ko_doro'] == road_name, results))

                if len(filtered) == 1:
                    a_id = filtered[0]['address_id']
                    res = jdbc.fetch_data(
                        AddressParser.__search_by_aid.format(search_columns),
                        (a_id, ),
                        ret_type='raw')
                    if len(res) == 1:
                        if additional_columns is None:
                            return res[0][0]
                        return res[0]

                res = jdbc.fetch_data(
                    AddressParser.__road_search_query.format(search_columns),
                    splitted_addr[:3] + (splitted_addr[-1], num1, num2),
                    ret_type='raw')
                if len(res) == 1:
                    if additional_columns is None:
                        return res[0][0]
                    return res[0]

                return None
            raise Exception('도로명 주소 서버 통신 실패')

        def search_jibun(addr, splitted_addr):
            '''
            주소 형태가 지번 주소인 경우라고 판단 할 때, 매칭하는 코드

            addr : 주소
            splitted_addr : 분리된 주소
            '''
            number = addr.split()[-1]
            if number[0] == '산':
                number = number[1:]
                is_mountain = 2
            else:
                is_mountain = 1
            number = number.split('-')
            num1 = number[0]
            num2_case = f'num2 = {number[1]}' if len(
                number) == 2 and number[1].isdigit() else '(num2 IS NULL or num2 = 0)'

            res = jdbc.fetch_data(AddressParser.__jibun_search_query.format(
                search_columns, num2_case),
                                  splitted_addr + (is_mountain, num1),
                                  ret_type='raw')
            if len(res) == 1:
                if additional_columns is None:
                    return res[0][0]
                return res[0]
            return None

        match = re.compile(r'\s+산{0,1}\d{1,}-?\d{0,}').search(addr)

        if match is None:
            return None

        used_addr = addr[:match.end()]

        road_splited_addr = self.parse(used_addr, how='road')
        jibun_splited_addr = self.parse(used_addr, how='jibun')
        addr_type = 'road'
        if road_splited_addr[-1] == '':
            addr_type = 'jibun'

        if addr_type == 'road':
            result = search_road(used_addr, road_splited_addr)
            if result is None:
                result = search_jibun(used_addr, jibun_splited_addr)
        else:
            result = search_jibun(used_addr, jibun_splited_addr)
            if result is None:
                result = search_road(used_addr, road_splited_addr)
        if result is None and not silent:
            print(f'"{used_addr}" 주소 하나로 결정 불가')
        return result
